In [1]:
from pony.orm import db_session, select
from be import db
from be.models import User, Course, Test, Question, Option, Topic, UserAnswer
import json

## Load seed data into db

In [2]:
db.bind(
    provider = 'mysql',
    host = 'localhost',
    user = 'its_admin',
    password = 'test123',
    db ='its-mysql'
)
db.generate_mapping(create_tables=True)

In [3]:
with open('seed/courses_list.json', 'rb') as seed_data:
    seed_data = json.load(seed_data)
    with db_session:
        for course in seed_data.keys():
            topics = []
            for t in seed_data[course]:
                topics.append(Topic(topic=t))
            Course(name=course, topics=topics)